In [3]:
from numpy import genfromtxt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import codecs
import numpy as np
import os
import shutil
import torch
import yaml
from datetime import datetime
from torch.utils.data.sampler import RandomSampler
from torch.autograd import Variable


In [4]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [5]:
train = data.iloc[0:1611]
test = data[1611:]
print(train.shape)
print(test.shape)

(1611, 27)
(378, 27)


In [6]:
trainheadlines = data.iloc[0:1611, 2:27].values
trainLabels = data.iloc[0:1611, 1].values
testheadlines = data.iloc[1611:, 2:27].values
testLabels = data.iloc[1611:, 1].values

print(trainheadlines.shape)
print(trainLabels[:10])

(1611, 25)
[0 1 0 0 1 1 0 0 1 1]


In [7]:
trainVocab = []
for row in range(0,len(train.index)):
    trainVocab.append(' '.join(str(x) for x in train.iloc[row,2:27]))

vectorizer = CountVectorizer()
basictrain = vectorizer.fit_transform(trainVocab)
print(basictrain.shape)

(1611, 31675)


In [8]:
trainheadlines = []
trainLabels = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
    trainLabels.append(int(train.iloc[row,1]))

basicvectorizer = CountVectorizer(min_df=3)
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

testheadlines = []
testLabels = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    testLabels.append(test.iloc[row,1])

basictest = basicvectorizer.transform(testheadlines)

(1611, 14073)


In [38]:
def transformHeadlines(headlines):
    outputHeadlines = np.empty([headlines.shape[0], 5])
    for i, headline in enumerate(headlines):
        outputHeadline = [vectorizer.vocabulary_[x] if x in vectorizer.vocabulary_ else -1 for x in vectorizer.build_tokenizer()(headline.lower())]
        outputHeadlines[i] = outputHeadline[:5]
        
    return np.asarray(outputHeadlines)

In [379]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf

class LSTMLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LSTMLayer, self).__init__()
        self.output_dim = output_dim
        self.lstmLayers = []
        for i in range(25):
            self.lstmLayers.append(nn.LSTM(input_dim, output_dim))
        self.basicLSTM = nn.LSTM(input_dim, output_dim)
        self.fc = nn.Linear(output_dim, 2)

    def forward(self, headlines, hidden = None, cell = None):
        outputs = []
        new_hiddens = []
        if hidden is None:
            hidden = Variable(torch.zeros(1, 1, self.output_dim)).float()
            cell = Variable(torch.zeros(1, 1, self.output_dim)).float()
        hidden = hidden.cuda()
        cell = cell.cuda()
        headlinesTransformed = Variable(torch.from_numpy(np.asarray(headlines.todense().astype(float)))).view(1,1,-1).float().cuda();
        output, (hidden, cell) = self.basicLSTM(headlinesTransformed,
                                                (hidden, cell))
        log_probs = F.log_softmax(self.fc(output[0]), dim=-1)
        return log_probs, hidden, cell


In [1]:
print(basicvectorizer.inverse_transform(basictrain[0]))


NameError: name 'basicvectorizer' is not defined

In [445]:
# from datetime import datetime

lstmLayer = LSTMLayer(basictrain[0].shape[1], 100).cuda()
# predictLayer = PredictLayer(30).cuda()

learning_rate = 0.1
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=learning_rate)
#predictOptimizer = torch.optim.Adam(predictLayer.parameters(), lr=learning_rate)
epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()

while epoch < 200:
    cell = None
    hidden = None
    total_loss = 0
    for (i, headlines) in enumerate(basictrain):
#         if i == 500:
#             break
        lstmOptimizer.zero_grad()
        log_probs, hidden, cell = lstmLayer(headlines, hidden, cell)
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
        
    lstmOptimizer.step()
    train_loss.append(total_loss.data)
    if ((i) % 10) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)), end='\r')
#         print('')
    epoch = epoch + 1

In [431]:
train_loss

[
  1135.1416
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  5414.7842
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1375.4298
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1191.9204
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1134.5706
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1126.3152
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1113.2526
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1077.9924
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1043.4398
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1085.4072
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1048.7307
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  1016.1223
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  956.4117
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  943.6127
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  911.4258
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  897.6584
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  899.7159
 [torch.cuda.FloatTensor of size 1 (GPU 0)], 
  910.7770
 [torc

In [452]:
state_dict = {
    'epoch': epoch,
    'lstmLayer': lstmLayer.state_dict(),
    'predictLayer': predictLayer.state_dict(),
    'lstm_optimizer': lstmOptimizer.state_dict(),
    'predict_optimizer': predictOptimizer.state_dict()
}
ckpt_path = 'data\ckpt\basic_most\model.pt'
# torch.save(state_dict, '/data/ckpt/basic_most/model.pt')
torch.save(state_dict, 'basic_most_model_working_epoch_200_lr_01_hidden_100.pt')

In [449]:

trainPredictions = []
log_probs = []
h = None
c = None
for (i, headlines) in enumerate(basictrain):
        log_prob, h, c = lstmLayer(headlines, h, c)
        log_probs.append(log_prob)
        if float(log_prob[0][0]) > float(log_prob[0][1]):
            trainPredictions.append(0)
        else:
            trainPredictions.append(1)

In [393]:
Counter(trainPredictions)

Counter({0: 232, 1: 268})

In [450]:

testPredictions = []
log_probs = []
# h = hidden
# c = cell
for (i, headlines) in enumerate(basictest):
        log_prob, h, c = lstmLayer(headlines, h, c)
        log_probs.append(log_prob)
        if float(log_prob[0][0]) > float(log_prob[0][1]):
            testPredictions.append(0)
        else:
            testPredictions.append(1)

In [451]:
print('Training Accuracy:', sum(np.asarray(trainPredictions) == np.asarray(trainLabels))/len(trainLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(trainLabels, trainPredictions))

print('Test Accuracy:', sum(np.asarray(testPredictions) == np.asarray(testLabels))/len(testLabels))
print(confusion_matrix(testLabels, testPredictions))

Training Accuracy: 0.998137802607
[[737   1]
 [  2 871]]
Test Accuracy: 0.452380952381
[[ 72 114]
 [ 93  99]]


In [429]:
len(testPredictions)

378

In [442]:
cell

Variable containing:
(0 ,.,.) = 

Columns 0 to 6 
    404.1833     1.0000 -1493.9633  -476.9284  -382.5286     0.0000   437.7519

Columns 7 to 13 
   1359.5787    -0.0064     0.0000    -2.9580     1.0000     0.0000    56.6718

Columns 14 to 20 
     -0.0000   578.1794     1.0000    -0.0000    -0.0000     2.9995     0.5660

Columns 21 to 27 
      9.0044    -1.0000   -12.5238     0.0000    -1.0000     0.0000     1.9999

Columns 28 to 29 
     -0.0000     0.0000
[torch.cuda.FloatTensor of size 1x1x30 (GPU 0)]